Confirming Gupta net works the same in both pytorch and tf. 

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
import h5py

In [3]:
import tensorflow as tf

In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [5]:
from uatu.watchers import *

In [6]:
from scipy.ndimage import rotate

In [7]:
def test(model_init_fn, x_np,y_np, fname, cost_fn, training=False, adv = False):
    tf.reset_default_graph()
    

    x = tf.placeholder(tf.float32, [None, 256,256,1], name = 'x')
    y = tf.placeholder(tf.float32, [None,2])
    
    with tf.variable_scope('kappa_filters') as scope:
        preds = model_init_fn(x, training=training)
    #loss = cost_fn(y, preds)
    #preds = tf.layers.flatten(preds)
    #with tf.device('/cpu:0'):
    saver = tf.train.Saver()

    with tf.Session() as sess:
        saver.restore(sess, fname)
        #sess.run(tf.global_variables_initializer())

        #assert y_np.shape[0] == 1 , 'batchsize greater than 1'
        #samples = []
        feed_dict = {x: x_np, y:y_np}#, training: False}

        #preds_np, loss_np  = sess.run([preds, loss], feed_dict=feed_dict)
        preds_np  = sess.run(preds, feed_dict=feed_dict)

        #samples.append(preds_np)

    return preds_np#, loss_np#np.vstack(samples)

In [8]:
f = h5py.File('../data/UatuLightconeTraining.hdf5', 'r')
mean, std =  f.attrs['mean'], f.attrs['std']
f.close()

In [9]:
f = h5py.File('../data/UatuLightconeSameSeed.hdf5', 'r')
boxno = 2
test_X = np.array(f['Box%03d'%boxno]["X"][()])
test_Y = f['Box%03d'%boxno]['Y'][0].reshape((1,2))
f.close()

In [10]:
test_X = (test_X-mean)/(std)

In [11]:
n = 100 
batch_size = 1 

X = test_X[n, :, :, :].reshape((1, test_X.shape[1],\
                                test_X.shape[2], test_X.shape[3]))
X = np.tile(X, (batch_size, 1, 1, 1) )
Y = test_Y[0, :].reshape((1, test_Y.shape[1]))
Y = np.tile(Y, (batch_size, 1))

In [12]:
net = 'adversarial'
network_fname = '../networks/gupta_net_kappa_adv_abs-9050'
model_init_fn = gupta_adv_network_init_fn
cost_fn = adversarial_standard_abs_cost_fn
training=False
adv = True

In [13]:
pred = test(model_init_fn, X,Y, network_fname, cost_fn, training, adv=adv)

In [14]:
pred.shape

(1, 256)

In [15]:
pred[0,:10]

array([-0.00641268,  0.00758688, -0.00231059,  0.06186396, -0.00331707,
       -0.05586817,  0.02360274,  0.05106198, -0.0295847 ,  0.07001832],
      dtype=float32)

In [16]:
np.save('tmp.npy', pred)
#np.save('flat_out.npy', pred)
#np.save('weight_mat.npy', pred)

In [17]:
Y

array([[0.256755, 0.77129 ]])